Esse notebook faz a segunda análise dos dados da CEMIG.

# Importações, configurações e funções

In [12]:
import sys
sys.path.append('../')

In [13]:
import os
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
from sqlalchemy import MetaData
from sqlalchemy import exc
from io import StringIO
import table_columns
import re
import csv

In [14]:
# engine = create_engine('postgresql+psycopg2://{}:{}@{}:{}/{}'.format(
#     'dictasstage',
#     'IfZSK740mMNHrUbA',
#     '40.87.86.193',
#     '5432' ,
#     'dictasstage'
# ))

# %env SCHEMA = dictasstage
# operadora = 17

# def load_to_db_append(df):
#     df.to_sql('stg_op{}_{}'.format(operadora, tabela),
#           con=engine,
#           schema=os.getenv('SCHEMA'),
#           if_exists='append',
#           index=False,
#           chunksize=10000,
#           method='multi'
#          )

env: SCHEMA=dictasstage


In [ ]:
# engine = create_engine('postgresql+psycopg2://{}:{}@{}:{}/{}'.format(
#     'bunch',
#     'Softplan01',
#     'localhost',
#     '5432' ,
#     'dictasstage'
# ))

# %env SCHEMA = dictasstage
# operadora = 17

# def load_to_db_append(df):
#     df.to_sql('stg_op{}_{}'.format(operadora, tabela),
#           con=engine,
#           schema=os.getenv('SCHEMA'),
#           if_exists='append',
#           index=False,
#           chunksize=10000,
#           method='multi'
#          )

In [15]:
engine = create_engine('postgresql+psycopg2://{}:{}@{}:{}/{}'.format(
    'etl_user',
    'etl_user',
    '172.27.13.6',
    '5432' ,
    'dictas'
))

%env SCHEMA = dictas_stage
operadora = 17

def load_to_db_append(df):
    df.to_sql(tabela,
          con=engine,
          schema=os.getenv('SCHEMA'),
          if_exists='append',
          index=False,
          chunksize=10000,
          method='multi'
         )

env: SCHEMA=dictas_stage


In [4]:
def rename_columns(df, dict_rename, colunas):
    # Ananalise das colunas que sobraram
    colunas_sobraram = []
    for item in dict_rename:
        if dict_rename[item] == '':
            colunas_sobraram.append(item)
            # Remove coluna sobrante
            df.drop(item, axis=1, inplace=True)  
    if len(colunas_sobraram) > 0:
        print('Sobraram as colunas:', colunas_sobraram)
    
    # Renomeando colunas
    df.rename(inplace=True, columns=dict_rename)
    
    # Analise das colunas faltantes
    colunas_faltantes = [value for value in colunas if not value in df.columns]
    if len(colunas_faltantes) > 0:
        print('Faltaram as colunas:', colunas_faltantes)
    return df

In [5]:
def apply_strip_on_type_object(df):
    for column in df:
        if df[column].dtype == 'object':
            df[column] = df[column].astype(str).str.strip()
    return df

def force_to_int(string):
    try:
        return int(string)
    except ValueError:
        return None
    
def transform_to_datetime(dt_in_string, date_format='%d/%m/%Y'):
    if dt_in_string == '':
        return None
    else:
        try:
            df_in_datime = pd.to_datetime(dt_in_string, format=date_format)
            return df_in_datime
        except pd.errors.OutOfBoundsDatetime:
            return None 
        
def transform_ind_internacao(ind_internacao):
    """
    Transform tipo_pessoa(string)
    return bool or 2 (fora do padrão)
    """
    if ind_internacao == 'NÃO':
        return 'N'
    elif ind_internacao == 'SIM':
        return 'S'
    else:
        return 2        

In [6]:
path = '/home/arlenwass/Projetos/Dictas/dados/cemig/2019-11-08/'

In [7]:
pd.options.display.max_columns=50

# bt_beneficiario

In [ ]:
file_path = os.path.join(path,'BENEFICIARIO.csv' )
colunas = table_columns.bt_beneficiario
df = pd.read_csv(file_path, sep='|', error_bad_lines=False, header = None, quoting=csv.QUOTE_NONE)

dict_rename = {}
for i in range(len(df.columns)):
    dict_rename[df.columns[i]] =  colunas[i]

df = rename_columns(df, dict_rename, colunas)

In [ ]:
df.head()

In [ ]:
index_to_drop = df.query('nr_beneficiario.astype("str").str.contains("NULL")').index
df.iloc[index_to_drop].head(3)

In [ ]:
df.iloc[index_to_drop].shape[0]

In [ ]:
df[df.dt_nascimento == '0001\t'].head(3)

# bt_empresa

In [ ]:
file_path = os.path.join(path, 'EMPRESA.csv')
colunas = table_columns.bt_empresa
df = pd.read_csv(file_path, sep='|', error_bad_lines=False, header = None, quoting=csv.QUOTE_NONE, low_memory=False)

dict_rename = {}
for i in range(len(df.columns)):
    dict_rename[df.columns[i]] =  colunas[i]

df = rename_columns(df, dict_rename, colunas)

In [ ]:
df[df['pk_empresa'].duplicated(keep=False)].sort_values('pk_empresa').head()

# bt_medico_prestador

In [ ]:
file_path = os.path.join(path, 'PRESTADORES.csv')
colunas = table_columns.bt_medico_prestador
df = pd.read_csv(file_path, sep='|', error_bad_lines=False, header = None, quoting=csv.QUOTE_NONE, low_memory=False, na_values='NULL\t')

dict_rename = {}
for i in range(len(df.columns)):
    dict_rename[df.columns[i]] =  colunas[i]

df = rename_columns(df, dict_rename, colunas)

In [ ]:
df = df.drop_duplicates('pk_medico_prestador')

In [ ]:
df['dat_nascimento'] = df.dat_nascimento.str.replace('\t', '')

df['ano']= df.dat_nascimento.apply(lambda x: x[-4:] if x is not np.nan else x)

df[df['ano'] == '9175']

# bt_vidas

In [ ]:
file_path = os.path.join(path, 'vidas')
file_path

In [ ]:
vidas_files = os.listdir(file_path)

In [ ]:
colunas = table_columns.bt_vida
df = pd.read_csv(os.path.join(file_path, vidas_files[0]), sep='|', error_bad_lines=False, encoding='utf8', header = None, quoting=csv.QUOTE_NONE, low_memory=False)

dict_rename = {}
for i in range(len(df.columns)):
    dict_rename[df.columns[i]] =  colunas[i]

df = rename_columns(df, dict_rename, colunas)
df.head()

In [ ]:
list_of_sizes = []
for table in vidas_files:
    df = pd.read_csv(os.path.join(file_path, table), sep='|', error_bad_lines=False, encoding='utf8', header = None, quoting=csv.QUOTE_NONE, low_memory=False)
    list_of_sizes.append(df.shape[0])
    
sum(list_of_sizes)    

# bt_custo

In [16]:
tabela = 'bt_custo'
colunas = table_columns.bt_custo_cemig

In [17]:
files_path = os.path.join(path, 'Custos_atualizado')

In [18]:
for a in os.listdir(files_path):
    file = os.path.join(files_path, a)
    print(file)
    df = pd.read_csv(file, sep='|', encoding='utf8', header=None, error_bad_lines=False,
            quoting=csv.QUOTE_NONE, na_values='NULL\t')

    dict_rename = {}
    for i in range(len(df.columns)):
        dict_rename[df.columns[i]] =  colunas[i]

    df = rename_columns(df, dict_rename, colunas)
    df = apply_strip_on_type_object(df)
    df['ind_internacao'] = df.ind_internacao.apply(transform_ind_internacao)
    df['cod_medicoexecutante'] = df.cod_medicoexecutante.apply(force_to_int)
    df['cod_medicosolicitante'] = df.cod_medicosolicitante.apply(force_to_int)

    df = df.query('dt_realizacao != "/  /"')

    df['mes_competencia'] = df.mes_competencia.astype(str).str.rjust(6, '0')
    df['mes_competencia'] = df.mes_competencia.apply(transform_to_datetime, args=('%m%Y',))

    df['vl_co'] = df.vl_co.fillna(0)
    df['vl_hm'] = df.vl_hm.fillna(0)
    df['vl_filme'] = df.vl_filme.fillna(0)
    df['vl_total'] = df.vl_total.fillna(0)
    
    df['dt_realizacao'] = df.dt_realizacao.apply(transform_to_datetime)
    df['dt_ini_internacao'] = df.dt_ini_internacao.apply(transform_to_datetime)
    df['dt_fim_internacao'] = df.dt_fim_internacao.apply(transform_to_datetime)

    df['cod_operadora'] = 17

    load_to_db_append(df)

/home/arlenwass/Projetos/Dictas/dados/cemig/2019-11-08/Custos_atualizado/CUSTO 052017.csv
/home/arlenwass/Projetos/Dictas/dados/cemig/2019-11-08/Custos_atualizado/CUSTO 062019.csv
/home/arlenwass/Projetos/Dictas/dados/cemig/2019-11-08/Custos_atualizado/CUSTO 022019.csv
/home/arlenwass/Projetos/Dictas/dados/cemig/2019-11-08/Custos_atualizado/CUSTO 022018.csv
/home/arlenwass/Projetos/Dictas/dados/cemig/2019-11-08/Custos_atualizado/CUSTO 072017.csv
/home/arlenwass/Projetos/Dictas/dados/cemig/2019-11-08/Custos_atualizado/CUSTO 052018.csv
/home/arlenwass/Projetos/Dictas/dados/cemig/2019-11-08/Custos_atualizado/CUSTO 042018.csv
/home/arlenwass/Projetos/Dictas/dados/cemig/2019-11-08/Custos_atualizado/CUSTO 082019.csv
/home/arlenwass/Projetos/Dictas/dados/cemig/2019-11-08/Custos_atualizado/CUSTO 012018.csv
/home/arlenwass/Projetos/Dictas/dados/cemig/2019-11-08/Custos_atualizado/CUSTO 032019.csv
/home/arlenwass/Projetos/Dictas/dados/cemig/2019-11-08/Custos_atualizado/CUSTO 092019.csv
/home/arle

b'Skipping line 54626: expected 32 fields, saw 33\n'


/home/arlenwass/Projetos/Dictas/dados/cemig/2019-11-08/Custos_atualizado/CUSTO 072019.csv
/home/arlenwass/Projetos/Dictas/dados/cemig/2019-11-08/Custos_atualizado/CUSTO 062017.csv
/home/arlenwass/Projetos/Dictas/dados/cemig/2019-11-08/Custos_atualizado/CUSTO 042017.csv
/home/arlenwass/Projetos/Dictas/dados/cemig/2019-11-08/Custos_atualizado/CUSTO 032017.csv
/home/arlenwass/Projetos/Dictas/dados/cemig/2019-11-08/Custos_atualizado/CUSTO 052019.csv
/home/arlenwass/Projetos/Dictas/dados/cemig/2019-11-08/Custos_atualizado/CUSTO 072018.csv
/home/arlenwass/Projetos/Dictas/dados/cemig/2019-11-08/Custos_atualizado/CUSTO 012019.csv
/home/arlenwass/Projetos/Dictas/dados/cemig/2019-11-08/Custos_atualizado/CUSTO 082017.csv
/home/arlenwass/Projetos/Dictas/dados/cemig/2019-11-08/Custos_atualizado/CUSTO 092018.csv
/home/arlenwass/Projetos/Dictas/dados/cemig/2019-11-08/Custos_atualizado/CUSTO 092017.csv


In [ ]:
shapes = []
for a in os.listdir(files_path):
    file = os.path.join(files_path, a)
    df = pd.read_csv(file, sep='|', encoding='utf8', header=None, error_bad_lines=False,
            quoting=csv.QUOTE_NONE, na_values='NULL\t')
    shapes.append(df.shape[0])



In [19]:
print('acabou')

acabou


In [ ]:
sum(shapes)

In [ ]:
os.listdir(files_path)

# bt_servico

In [ ]:
file_path = os.path.join(path, 'SERVICOS.csv')
colunas = table_columns.bt_servico
df = pd.read_csv(file_path, sep='|', error_bad_lines=False, encoding='utf8', header = None, quoting=csv.QUOTE_NONE, low_memory=False)

dict_rename = {}
for i in range(len(df.columns)):
    dict_rename[df.columns[i]] =  colunas[i]

df = rename_columns(df, dict_rename, colunas)

In [ ]:
df.head()

# bt_consultas

In [ ]:
file_path = os.path.join(path,'CONSULTAS.csv' )
colunas = table_columns.bt_consulta
df = pd.read_csv(file_path, sep='|', error_bad_lines=False, header = None, quoting=csv.QUOTE_NONE)

dict_rename = {}
for i in range(len(df.columns)):
    dict_rename[df.columns[i]] =  colunas[i]

df = rename_columns(df, dict_rename, colunas)

In [ ]:
df.head()

# bt_diarias

In [ ]:
file_path = os.path.join(path, 'DIARIAS.csv')
colunas = table_columns.bt_diaria
df = pd.read_csv(file_path, sep='|', error_bad_lines=False, header = None, quoting=csv.QUOTE_NONE, low_memory=False)

dict_rename = {}
for i in range(len(df.columns)):
    dict_rename[df.columns[i]] =  colunas[i]

df = rename_columns(df, dict_rename, colunas)

In [ ]:
df.head()